In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt 

import napari
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from utils import  load_video_as_array, moving_average, array_seconds_to_minute_second
from scipy.stats import zscore
from consts import HERE

from two_photon import load_data 

In [2]:
pupil_umbrella = Path("/Volumes/MarcBusche/James/Regular2p")

# date = "2024-09-27"
# mouse = "J023"

# mouse = "J025"
# date = "2024-09-27"

mouse = "J024"
date = "2024-10-09"


pupil_folder = pupil_umbrella / date / mouse
mp4_paths = sorted(list(pupil_folder.glob("*.mp4")))

In [3]:
show_napari = True

if show_napari:
    # Inspect the binary form of the video
    video = load_video_as_array(mp4_paths[0], 500)

    viewer = napari.Viewer()
    binarised = (video > 200).astype(int)

    viewer.add_image(video, name="Video")
    viewer.add_image(binarised, name="Binarised")
    viewer.add_image(np.abs(np.diff(binarised, 2)), name="Diffed")

In [4]:
diffed = np.load(HERE.parent / "cache" /  f"{mouse}-{date}-diffed.npy")
movement = moving_average(zscore(diffed), 10)
# Add a value onto the end to get it the same shape as the dff
movement = np.append(movement, movement[-1])

In [5]:
plot_movement = False

if plot_movement:
    plt.figure()
    x_labels = array_seconds_to_minute_second(np.arange(len(movement)) / 30)
    plt.plot(movement)
    spacing = 60 
    plt.xticks(range(0, len(movement), spacing), x_labels[::spacing], rotation=90);
    plt.show()

In [6]:
dff, deconv = load_data(mouse, date)

# Pupil was not saved for the final recording so this is a valid maneuver for this mouse
if mouse == "J025" and date == "2024-09-27":
    dff = dff[:, :len(movement)]
    deconv = deconv[:, :len(movement)]

assert movement.shape[0] == deconv.shape[1]  == dff.shape[1]

In [7]:
# There are 32 frames that are nan here so the rate is very slightly off but not enough to make a difference

resting = movement < -0.5
print(f'percent resting: {np.sum(resting) / len(resting) * 100} %')

resting_spike_rate = np.nansum(deconv[:, resting], 1) / (np.sum(resting) / 30)
moving_spike_rate = np.nansum(deconv[:, ~resting], 1) / (np.sum(~resting) / 30)


np.save(HERE.parent / "cache" / f"{mouse}-{date}-resting-rates.npy", resting_spike_rate)
np.save(HERE.parent / "cache" / f"{mouse}-{date}-moving-rates.npy", moving_spike_rate)

print(f"Resting spike rate: {np.mean(resting_spike_rate)}") 
print(f"Moving spike rate: {np.mean(moving_spike_rate)}")

percent resting: 46.96666666666667 %
Resting spike rate: 3.5320412630977835
Moving spike rate: 3.6357470832200267
